In [1]:
import numpy as np
import pandas as pd
import re
import time
from glob import glob
from string import punctuation
from collections import Counter, OrderedDict

import requests
from requests_html import HTMLSession

# from string_grouper import match_strings, match_most_similar, group_similar_strings, StringGrouper
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse import csr_matrix
# import sparse_dot_topn.sparse_dot_topn as sdt

cosmetic ingredient database: https://public.opendatasoft.com/explore/dataset/cosmetic-ingredient-database-ingredients-and-fragrance-inventory/table/

cosdna: https://cosdna.com/

In [7]:
file_list = glob('../data/*.csv')
file_list = sorted(file_list)
file_list

['../data\\brands.csv',
 '../data\\recommendations_2020-05-05.csv',
 '../data\\responses_2020-05-05.csv',
 '../data\\responses_modified_2020-05-05.csv']

In [8]:
responses = pd.read_csv(file_list[-1])

In [10]:
responses.head()

,Timestamp,What's your skin type?,Do you have any known allergies? Or any ingredients you avoid for any reason?,What are your top skin concerns?,What is your current day routine? (Please list your products in order from first to last),What is your current night routine (Please list your products in order from first to last),What type of summer weather will you have?,Any known skin sensitivities?,"Any experience with retinoids? (If so, please tell us your use history)","Any experience with acids? (If so, please tell us your use history, what type, and any known concentrations)","Are you prone to breakouts? If so, how severe are the breakouts and what is your acne treatment history?",Anything else you want the chemists to know about your skin and your skincare routine?,"Can we share your routine learnings (anonymously) on our social media (post, podcast, or insta live shoutout)?"
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes
2,4/30/2020 12:50:26,Oily,No.,"Acne, Pigmentation","Drunk Elephant Beste No. 9 Jelly Cleanser, pau...","Dermalogica Precleanse, CeraVe Foaming Facial ...",Desert dry heat,No.,No.,I've used 2% BHA on and off; and daily I use a...,"Yes, occasional small whiteheads and blackhead...",I'm Black so hyperpigmentation and skin discol...,Yes
3,4/30/2020 13:02:43,Normal,No,"Fine lines/wrinkles, Skin Texture (roughness, ...","Paula's Choice enriched calming toner, Timeles...","Palmer's facial cleansing oil, Cetaphil gentle...","Tropical, humid",No,Almost through the purging/peeling phase with ...,I've tried (and still own) The Ordinary red pe...,"Not really - a very rare pimple, I tend more o...",I turn 43 in a month and am really trying to g...,Yes
4,4/30/2020 13:08:20,Combination,"Ethylhexyl palmitate, strong sulfates, chemica...","Acne, Skin Texture (roughness, dullness), Other","Osea Ocean Cleanser, Renee Rouleau Elderberry ...","Renee Rouleau Luxe Mint Cleanser, Renee Roulea...","Temperate, a lovely 70F (lucky!)",Overall I have fairly sensitive skin with horm...,"No, but I would like to start to address early...",Azelaic acid has been a good acid for me (not ...,"My acne is adult onset, primarily hormonal, an...","I just moved to Chicago, so lots of temperatur...",Yes


In [12]:
new_columns = [
    'timestamp',
    'skin_type',
    'allergies',
    'top_skin_concern',
    'am_routine',
    'pm_routine',
    'summer_climate',
    'skin_sensitivities',
    'used_retinoids',
    'used_acids',
    'prone_to_breakouts',
    'miscellaneous',
    'permission'
]

responses.columns = new_columns
# responses.drop(columns=['email_address','name'], inplace=True)
responses.head()

,timestamp,skin_type,allergies,top_skin_concern,am_routine,pm_routine,summer_climate,skin_sensitivities,used_retinoids,used_acids,prone_to_breakouts,miscellaneous,permission
0,4/30/2020 12:43:32,Combination,None,"Acne, Pigmentation, Fine lines/wrinkles","trader joe's all in one cleanser, bioderma seb...","trader joe's all in one cleanser, Bioderma Seb...","Temperate, a lovely 70F (lucky!)",I broke out from Cosrx snail serum and from Ma...,Differing 0.1% for four years,I prefer toners with acids as masks with AHA s...,I have mild to moderate acne that I have been ...,"I have a super oily forehead, oily nose and c...",Yes
1,4/30/2020 12:45:25,Combination,No allergies,"Acne, Fine lines/wrinkles, Skin Texture (rough...","Biore Charcoal Acne Clearing Cleanser, Kiel's ...",NaN,"Tropical, humid",No.,No.,No.,Yes. They are usually cystic and around my cyc...,I need help.,Yes
2,4/30/2020 12:50:26,Oily,No.,"Acne, Pigmentation","Drunk Elephant Beste No. 9 Jelly Cleanser, pau...","Dermalogica Precleanse, CeraVe Foaming Facial ...",Desert dry heat,No.,No.,I've used 2% BHA on and off; and daily I use a...,"Yes, occasional small whiteheads and blackhead...",I'm Black so hyperpigmentation and skin discol...,Yes
3,4/30/2020 13:02:43,Normal,No,"Fine lines/wrinkles, Skin Texture (roughness, ...","Paula's Choice enriched calming toner, Timeles...","Palmer's facial cleansing oil, Cetaphil gentle...","Tropical, humid",No,Almost through the purging/peeling phase with ...,I've tried (and still own) The Ordinary red pe...,"Not really - a very rare pimple, I tend more o...",I turn 43 in a month and am really trying to g...,Yes
4,4/30/2020 13:08:20,Combination,"Ethylhexyl palmitate, strong sulfates, chemica...","Acne, Skin Texture (roughness, dullness), Other","Osea Ocean Cleanser, Renee Rouleau Elderberry ...","Renee Rouleau Luxe Mint Cleanser, Renee Roulea...","Temperate, a lovely 70F (lucky!)",Overall I have fairly sensitive skin with horm...,"No, but I would like to start to address early...",Azelaic acid has been a good acid for me (not ...,"My acne is adult onset, primarily hormonal, an...","I just moved to Chicago, so lots of temperatur...",Yes


In [15]:
# responses['top_skin_concern'] = responses['top_skin_concern'].str.lower().str.split(', ')
responses['top_skin_concern'][:5]

0            [acne, pigmentation, fine lines/wrinkles]
1    [acne, fine lines/wrinkles, skin texture (roug...
2                                 [acne, pigmentation]
3    [fine lines/wrinkles, skin texture (roughness,...
4    [acne, skin texture (roughness, dullness), other]
Name: top_skin_concern, dtype: object

## using requests-html

https://www.jcchouinard.com/web-scraping-with-python-and-requests-html/

In [16]:
aliases = {}
base_names = set()

In [ ]:
# test = 'HELLO'
# test_lower = test.lower
# test = test_lower()
# print(test)
# test = []
# test_append = test.append
# test_append(5)
# print(test)

In [17]:
class OrderedCounter(Counter, OrderedDict):
    pass

In [76]:
class CosDNA(HTMLSession):
    '''
    Parent class for connecting to CosDNA.com database.
    Not intended to be used on its own.
    '''
    
    base_url = 'https://cosdna.com'
    
    sort_dict = {
        'latest': '&sort=date',
        'featured': '&sort=featured',
        'clicks': '&sort=click',
        'reviews': '&sort=review'
    }
    
    def __init__(self, name=None):
        super().__init__()
        self._name = name
        self._synced = False      # synced in child classes

In [106]:
class Cosmetic(CosDNA):
    '''
    Parent class for organizing Products and Ingredients from CosDNA.com.
    Not intended to be used on its own.
    '''
    def __init__(self, name=None, cosdna_url=None):
        super().__init__(name)
        self._cosdna_url = cosdna_url
            
    def link(self, sort=None, cosdna_url=None, _base_url=None):
        self._cosdna_url = self._check_url(cosdna_url)
        if self._cosdna_url:
            return self
        else:
            self._search(sort, _base_url)
            
    def _search(self, sort=None, _base_url=None):
        if self._query:
            search_url = self._get_search_url(sort, _base_url)
            self._r = self.get(search_url)
            top = self._r.html.find('td', first=True)    # top result
            if top:
                self._cosdna_url = (CosDNA.base_url 
                                    + top.xpath('//a/@href', first=True))
                return self
            else:
                nr = self._r.html.find('.text-danger')
                if nr:
                    print(f'No results for {self._query} on CosDNA.')
                    print('Enter new search:')
                    name = input("(to cancel, enter 'BREAK' w/o quotes)")
                    if name == 'BREAK':
                        return self
                    else:
                        self._query = name
                        return self._search(sort, _base_url)
                else:
                    self._cosdna_url = self._r.url
                    return self
        else:
            print('Link with valid CosDNA URL or product name to proceed.')
            return self
    
    def _get_search_url(self, sort=None, _base_url=None):
        query = re.sub('[^a-z0-9\s\-]', '', self._query.lower())
        query = re.sub('([a-z])\-([a-z])', r'\1 \2', query)
        query = query.replace(' ', '+')
        # _base_url defined in child classes
        # Product() has different sort options, Ingredient() does not
        if sort in [*CosDNA.sort_dict]:
            search_url = _base_url + query + CosDNA.sort_dict[sort]
        else:
            search_url = _base_url + query
        return search_url
    
    def sync(self):
        if self.cosdna_url:
            # child classes define more actions
            self._r = self.get(self.cosdna_url)
            return self
        else:
            print('Initialize or link with valid CosDNA URL to proceed')
            return self        
    
    def _check_url(self, url=None):
        if not url:
            if not self._cosdna_url:
                return None
            else:
                return self._cosdna_url
        elif CosDNA.base_url in url:
            return url
        else:
            print('Invalid CosDNA URL.')
            return None
        
    @property
    def cosdna_url(self):
        return self._check_url(self._cosdna_url)
    
    @property
    def linked(self):
        if self.cosdna_url:
            return True
        else:
            return False
        
    @property
    # defined as a property since Routine() will not share this behavior
    def synced(self):
        return self._synced

In [129]:
class Ingredient(Cosmetic):
    '''
    Sync and store ingredient information from CosDNA.com.
    
    Search for ingredients based on name, link to CosDNA page, and scrape 
    information to store in instance.
    
    Parameters
    ----------
    name : str, default None
        Name of ingredient.
        
    cas_no : str, default None
        CAS Registry Number
        
    cosdna_url : str, default None
        URL of ingredient in CosDNA database
        
    >>> i = Ingredient('salicylic acid')
    >>> i.name                                     # returns assigned name
    'salicylic acid'
    >>> i.link_sync()                              # scrapes top result
    >>> i.name                                     # returns CosDNA name
    'bha'
    
    >>> i.link('https://cosdna.com/eng/0f1b7f1402.html')    # directly update link
    >>> i.sync()
    >>> i.name
    'capryloyl salicylic acid'
    '''
    
    def __init__(self, name=None, cas_no=None, cosdna_url=None):
        self.cas_no, self._cosdna_id = cas_no, None
        super().__init__(name, cosdna_url)
        
    def link(self, cosdna_url=None):
        if self.cas_no:
            self._query = self.cas_no
        else:
            self._query = self._name
        return super().link(sort=None, cosdna_url=cosdna_url, 
                            _base_url='https://cosdna.com/eng/stuff.php?q=')
        
    def sync(self):
        super().sync()     # goes to cosdna_url
        self._cosdna_name, self.aliases = self._get_names()
        self.mass, self.hlb, self.cas_no = self._get_chemical_info()
        self.description = self._get_description()
        self._synced = True
        return self
        
    def link_sync(self, cosdna_url=None):
        self.link(cosdna_url)
        self.sync()
        return self
    
    def _get_names(self):
        cosdna_name = self._r.html.find('.text-vampire', first=True) \
                          .text.lower()
        aliases = self._r.html.find('div.chem.mb-5 > div.mb-2', first=True) \
                      .text.lower().split(', ')
        return cosdna_name, aliases
    
    def _get_chemical_info(self):
        mass, hlb, cas_no = None, None, None
        ci = self._r.html                                                \
                 .find('div.d-flex.justify-content-between', first=True) \
                 .text
        if 'Molecular Weight' in ci:
            try:
                mass = float(re.findall(".*Weight[^\d\.]+(\d+\.\d+).*", ci)[0])
            except:
                mass = None
        if 'HLB' in ci:
            try:
                hlb = float(re.findall(".*HLB[^\d\.]+(\d+\.\d+).*", ci)[0])
            except:
                hlb = None
        if 'Cas No' in ci:
            cas_no = re.findall(".*Cas No[^\d\-]+(\d+\-\d+\-\d+).*", ci)[0]
        return mass, hlb, cas_no
    
    def _get_description(self):
        return self._r.html                                                  \
                   .find('div.chem.mb-5 > div.linkb1.ls-2.lh-1', first=True) \
                   .text
        
    @property
    def name(self):
        if self.synced:
            return self._cosdna_name
        else:
            return self._name
        
    @property
    def cosdna_id(self):
        if not self._cosdna_id:
            if self.cosdna_url:
                self._cosdna_id = re.findall(
                    "eng/(.*).html", self.cosdna_url)[0]
            else:
                self._cosdna_id = 'unavailable'
        return self._cosdna_id
        
    def __str__(self):
        return self.name

In [108]:
def ingredient_test(ing):
    tick = time.time()
    print(ing.linked)
    ing.link_sync()
    print('--link_synced')
    print(ing.linked)
    print()
    print(ing.name)
    print(ing.aliases)
    print(ing.cosdna_id)
    print(ing.mass)
    print(ing.hlb)
    print(ing.cas_no)
    print(ing.description)
    tock = time.time()
    print()
    print(tock - tick)

In [109]:
test = Ingredient('polysorbate-20')

ingredient_test(test)

False
--link_synced
True

tween 20
['poe (20) sorbitan monolaurate', 'polysorbate 20', 'atmer 110', 'peg(20)sorbitan monolaurate', 'alkest tw 20']
62e334964
1227.54
16.7
9005-64-5
Surfactant, Emulsifier

0.26927947998046875


In [93]:
test = Ingredient(cas_no='9005-64-5')

ingredient_test(test)

False
query = 9005-64-5
https://cosdna.com/eng/62e334964.html
--link_synced
True

tween 20
['poe (20) sorbitan monolaurate', 'polysorbate 20', 'atmer 110', 'peg(20)sorbitan monolaurate', 'alkest tw 20']
62e334964
1227.54
16.7
9005-64-5
Surfactant, Emulsifier

0.41489458084106445


In [94]:
test = Ingredient(cosdna_url='https://cosdna.com/eng/62e334964.html')

ingredient_test(test)

True
--link_synced
True

tween 20
['poe (20) sorbitan monolaurate', 'polysorbate 20', 'atmer 110', 'peg(20)sorbitan monolaurate', 'alkest tw 20']
62e334964
1227.54
16.7
9005-64-5
Surfactant, Emulsifier

0.2184145450592041


In [130]:
class Product(Cosmetic):
    '''
    Sync and store product information from CosDNA.com.
    
    Search for products based on name, link to CosDNA page, and scrape 
    information to store in instance.
    
    Parameters
    ----------
    name : str, default None
        Name of ingredient.
        
    brand : str, default None
        Name of brand. Does not affect search--purely for internal purposes.
        
    product : str, default None
        Name of product. Does not affect search--purely for internal purposes.
        
    cosdna_url : str, default None
        URL of ingredient in CosDNA database
    '''
    
    def __init__(self, name=None, brand=None, product=None, cosdna_url=None):
        self._name, self.brand, self.product = name, brand, product
        # initialize using 'name' property
        super().__init__(self.name, cosdna_url)
    
    def link(self, sort='featured', cosdna_url=None):
        # initialize using 'name' property
        self._query = self.name
        return super().link(sort=sort, cosdna_url=cosdna_url, 
                            _base_url='https://cosdna.com/eng/product.php?q=')
    
    def sync(self, deep=False):
        super().sync()
        self._set_name_brand_product(self._query)
        self._ingredients = self._get_ingredients(deep)
        self._synced = True
        return self
    
    def link_sync(self, sort='featured', cosdna_url=None, deep=False):
        self.link(sort=sort, cosdna_url=cosdna_url)
        self.sync(deep=deep)
        
    def _set_name_brand_product(self, name):
        cosdna_brand = self._r.html.find('.brand-name', first=True).text.lower()
        cosdna_product = self._r.html.find('.prod-name', first=True).text.lower()
        cosdna_name = str(cosdna_brand + ' ' + cosdna_product).strip()
        if cosdna_brand:
            self.brand, self.product = cosdna_brand, cosdna_product
        else:
            self._name = cosdna_name
            
    def _get_ingredients(self, deep):
        ingredients = []
        table = self._r.html.find('.tr-i')
        # not sure if this improves performance
        ingredients_append = ingredients.append
        for row in table:
            cells = row.find('td')
            if len(cells) == 5:
                ing, fun, _, _, _ = cells
                ing_name, ing_url = self._get_ingredient_info(ing)
                # figure out where to put function later
                # function = self._get_function_info(fun)
                ingredient = Ingredient(name=ing_name,
                                        cosdna_url=ing_url)
                if deep:
                    ingredient.sync()
                    time.sleep(0.5)
            else:
                ing = cells[0]
                ing_name = ing.find('.text-muted', first=True).text.strip() \
                              .lower()
                ingredient = Ingredient(name=ing_name)
            ingredients_append(ingredient)
        return ingredients            
            
    def _get_ingredient_info(self, ing):
        name = ing.text.strip().lower()
        url = ing.xpath('//a/@href', first=True)
        url = Cosmetic.base_url + url
        return name, url 
        
    def _get_function_info(self, fun):
        function = fun.text.strip().lower().split(',')
        if 'sunscreen' in function:
            try:
                uva = re.search("uv[ab]\d", 
                                fun.xpath('//img')[0].attrs['src'])[0]
                uvb = re.search("uv[ab]\d", 
                                fun.xpath('//img')[1].attrs['src'])[0]
                function.append(uva)
                function.append(uvb)
            except:
                pass
        return function    
    
    @property
    def name(self):
        if self.brand is None and self.product is None:
            return self._name
        else:
            self._name = self.brand + ' ' + self.product
            return self._name
    
    @property
    def ingredients(self):
        return [ing.name for ing in self._ingredients]
    
    @property
    def cosdna_urls(self):
        return [ing.cosdna_url for ing in self._ingredients]
    
    # _cosdna_ids and _ing_dict used for Routine()
    @property
    def _cosdna_ids(self):
        return [ing.cosdna_id for ing in self._ingredients]
    
    @property
    def _ingredient_dict(self):
        return dict(zip(self._cosdna_ids, self.ingredients))
        
    def __str__(self):
        return f'{self.name}\n\n{self.ingredients}'

In [111]:
def product_test(product):
    tick = time.time()
    print(product.linked)
    product.link_sync()
    print('--link_synced')
    print(product.linked)
    print()
    print(product.name)
    print(product.brand)
    print(product.product)
    print(product.ingredients)
    print(product._cosdna_ids)
    tock = time.time()
    print()
    print(tock - tick)

In [112]:
test = Product(brand='chemist confessions', product='better oil')
product_test(test)

False
--link_synced
True

chemist confessions better oil
chemist confessions
better oil
['hippophae rhamnoides seed oil', 'ribes nigrum seed oil', 'rosa rubiginosa seed oil', 'squalane', 'bisabolol', 'linoleic acid', 'linolenic acid', 'α-tocopherol', 'pelargonium graveolens oil', 'salvia sclarea oil', 'chamaecyparis obtusa oil']
['18c16716561', '5358a014437', '8a961d14427', '828b29359', 'affbc2955', '0287c3229', 'ba78081902', '3bf28a485', '12af7d17919', '4da61720318', '3f6f9914103']

0.9345018863677979


In [113]:
product2 = Product('chemist confessions better oil')
product_test(product2)

False
--link_synced
True

chemist confessions the better oil moisture basics nourishing emollients
None
None
['hippophae rhamnoides seed oil', 'ribes nigrum seed oil', 'rosa rubiginosa seed oil', 'squalane', 'bisabolol', 'linoleic acid', 'linolenic acid', 'α-tocopherol', 'pelargonium graveolens oil', 'salvia sclarea oil', 'chamaecyparis obtusa oil']
['18c16716561', '5358a014437', '8a961d14427', '828b29359', 'affbc2955', '0287c3229', 'ba78081902', '3bf28a485', '12af7d17919', '4da61720318', '3f6f9914103']

0.8706705570220947


In [114]:
product3 = Product(cosdna_url='https://cosdna.com/eng/cosmetic_57b4385905.html')
product_test(product3)

True
--link_synced
True

chemist confessions aquafix moisture basics hydrating water gel
None
None
['water', 'propanediol', 'ethoxydiglycol', 'glycerin', 'sodium pca', 'panthenol', 'allantoin', 'peg/ppg-17/6 copolymer', 'vinyl dimethicone/methicone silsesquioxane crosspolymer', 'madecassoside', 'asiaticoside', 'sodium hyaluronate', 'xanthan gum', 'lecithin', 'sclerotium gum', 'pullulan', 'polyacrylate crosspolymer-6', 'hexanediol', 'caprylhydroxamic acid', 'disodium edta']
['90172810251', '72a94a19514', 'b3d7c210384', '205eb9166', 'dfa882349', 'fc86a9293', 'f124da42', '1c459c8554', '293f3810714', 'd9fefd10987', 'ee7e9012948', '6b1f88346', '05a3b6831', 'cebeb8225', '7cef8d335', 'c7fedc6466', 'eba83f20507', '631f7f16507', 'eb396d14245', 'c25e5833']

0.27526187896728516


In [149]:
class Routine(CosDNA):
    
    def __init__(self, name=None, routine=None):
        super().__init__(name)
        self._routine = []
        if routine:
            self.add(routine)
        
    def add(self, routine):
        routine = np.array([routine])
        routine = routine.ravel()
        for product in routine:
            if type(product) == Product:
                self._routine.append(product)
            else:
                self._routine.append(Product(product))
        return self
    
    def link(self, sort='featured', force=False):
        self.link_sync(sort=sort, force=force, _link=True, _sync=False)
    
    def sync(self, force=False, deep=False):
        self.link_sync(sort=sort, force=force, deep=deep, _link=False, 
                       _sync=True)
    
    def link_sync(self, sort='featured', force=False, deep=False, _link=True, 
                  _sync=True):
        for product in self._routine:
            if _link:
                if force:
                    product.link(sort)
                else:
                    if not product.linked:
                        product.link(sort)
            if _sync:
                if force:
                    product.sync(deep)
                    changes = True
                else:
                    if not product.synced:
                        product.sync(deep)
                        changes = True
            time.sleep(0.5)
        if changes:
            self._analyze()
        else:
            return self
        
    def _analyze(self):
        """
        Tabulates frequency of ingredients across all Products
        """
#         self._rout_ids, self._rout_ings, self._rout_dict = self._get_routine_info()
        self._routine_ids, self._routine_dict = self._get_routine_info()
        self._counts = self._translate_counter(self._routine_dict,
                                               Counter(self._routine_ids))
        self._product_vectors = self._get_product_vectors()
        return self
    
    def _get_routine_info(self):
        routine_ids = []
#         routine_ings = []
        routine_dict = {}
        for product in self._routine:
            product_dict = product._ingredient_dict
            routine_ids += product._cosdna_ids
#             routine_ings += prod.ingredients
            routine_dict.update(product_dict)
        routine_dict.update({'unavailable': 'unavailable'})
        return routine_ids, routine_dict
#         return rout_ids, rout_ings, rout_dict
    
    def _translate_counter(self, translation_dict, counter):
        # https://stackoverflow.com/questions/51423217/
        return OrderedCounter(dict(
            (translation_dict.get(k, k), v) for (k, v) in counter.items()
        ))
    
    def _get_product_vectors(self):
        product_vectors = []
        for product in self._routine:
            product_vector = []
            product_vector_append = product_vector.append
            for cosdna_id in self._routine_ids:
                if cosdna_id in product._cosdna_ids:
                    product_vector_append(1)
                else:
                    product_vector_append(0)
            product_vectors.append(product_vector)
        return product_vectors
    
    def top_ingredients(self, top=None, mask=None):
        if mask:
            mask_count = OrderedCounter([x for x in self._ingredients if x in mask])
            return mask_count.most_common(top)
        else:
            return self._counts.most_common(top)
    
    def has(self, ingredient):
        ingredient_id = Ingredient(ingredient).link_sync().cosdna_id
        isolated_products = []
        for i, product_vector in enumerate(self._product_vectors):
            try:
                ingredient_index = self._routine_ids.index(ingredient_id)
                if product_vector[ingredient_index] == 1:
                    isolated_products.append(self._routine[i].name)
            except:
                print(f'Routine does not have {ingredient}.')
                break
        return isolated_products
    
    @property
    def routine(self):
        try:
            return [product.name for product in self._routine]
        except:
            return self._routine
        
    @property
    def products(self):
        return self.routine
    
    @property
    def cosdna_urls(self):
        return [product.cosdna_url for product in self._routine]
    
    @property
    def brands(self):
        return [product.brand for product in self._routine]
    
    @property
    def linked(self):
        try:
            return all([product.linked for product in self._routine])
        except:
            return False
    
    @property
    def synced(self):
        try:
            return all([product.synced for product in self._routine])
        except:
            return False
        
    @property
    def ingredients(self):
        return list(set(self._routine_ingredients))
    
    @property
    def top(self):
        return self.top_ingredients(10)
        
#     def __str__(self):
#         return f'Routine "{self.name}" with {len(self.routine)} products'
    
#     def __repr__(self):
#         return f'Routine(name={self.name}, routine={[product.name for product in self.routine]})'

In [150]:
tick = time.time()
routine_list = [
    'Biore Charcoal Acne Clearing Cleanser', 
    "kiehl's powerful strength line-reducing concentrate 12.5", 
    'Neutrogena Rapid Clear Spot Gel',
    'Roc Retinol Correxion eye cream', 
    'Body Merry Retinol Surge Moisturizer'
]


routine = Routine(routine=routine_list)
routine.link_sync()
tock = time.time()
print(tock - tick)

5.353680849075317


In [117]:
routine.top

[('glycerin', 4),
 ('disodium edta', 3),
 ('aqua', 3),
 ('ethylhexylglycerin', 2),
 ('sodium hydroxide', 2),
 ('unavailable', 2),
 ('glyceryl stearate', 2),
 ('panthenol', 2),
 ('polysorbate 20', 2),
 ('retinol', 2)]

In [157]:
tick = time.time()
routine.has('water')
tock = time.time()
print(tock-tick)

0.21445441246032715


In [152]:
routine.has('salicylic acid')

['biore charcoal acne clearing cleanser']

In [55]:
routine.routine

['biore charcoal acne clearing cleanser',
 "kiehl's powerful-strength line-reducing concentrate 12.5% vitamin c",
 'neutrogena rapid clear stubborn acne spot gel',
 'roc retinol correxion® eye cream 0.5oz(15ml)',
 'body merry retinol surge moisturizer']

In [119]:
routine.top

[('glycerin', 4),
 ('disodium edta', 3),
 ('aqua', 3),
 ('ethylhexylglycerin', 2),
 ('sodium hydroxide', 2),
 ('unavailable', 2),
 ('glyceryl stearate', 2),
 ('panthenol', 2),
 ('polysorbate 20', 2),
 ('retinol', 2)]

In [126]:
def routine_test(routine):
    
    tick = time.time()
    print(routine.routine)
    routine.link_sync()
    print()
    print(routine.routine)
    print(routine.has('water'))
    print(routine.top_ingredients(5))
    routine.add('la roche posay antihelios mineral')
    routine.link_sync()
    print()
    print(routine.top_ingredients(5))
    tock = time.time()
    print()
    print(tock - tick)

In [146]:
routine_list = [
    'Grown Alchemist Hydra-Restore Cream cleanser', 
    'Frudia Citrus Brightening Toner', 
    'Tatch The Essence', 
    'Peter Thomas Roth Retinol Fusion PM Serum'
]
routine = Routine(routine=routine_list)
routine_test(routine)

['Grown Alchemist Hydra-Restore Cream cleanser', 'Frudia Citrus Brightening Toner', 'Tatch The Essence', 'Peter Thomas Roth Retinol Fusion PM Serum']

['grown alchemist hydra-restore cream cleanser', 'frudia citrus brightening toner', 'tatcha the essence 150 ml | 5.1 fl. oz.', 'peter thomas roth retinol fusion pm night serum(a']
['grown alchemist hydra-restore cream cleanser', 'frudia citrus brightening toner', 'tatcha the essence 150 ml | 5.1 fl. oz.', 'peter thomas roth retinol fusion pm night serum(a']
[('water', 4), ('sorbitan olivate', 2), ('tocopherol', 2), ('niacinamide', 2), ('phenoxyethanol', 2)]

[('water', 5), ('tocopherol', 3), ('phenoxyethanol', 3), ('sorbitan olivate', 2), ('zinc oxide 5% (sunscreen)', 2)]

8.234978675842285


In [143]:
routine.has('vitamin e')

['grown alchemist hydra-restore cream cleanser',
 'frudia citrus brightening toner',
 'la roche-posay antihelios mineral sunscreen spf 50']

In [139]:
routine.has('water')

['grown alchemist hydra-restore cream cleanser',
 'frudia citrus brightening toner',
 'tatcha the essence 150 ml | 5.1 fl. oz.',
 'peter thomas roth retinol fusion pm night serum(a',
 'la roche-posay antihelios mineral sunscreen spf 50']

In [148]:
routine.has('salicylic acid')

Routine does not have salicylic acid


[]

In [156]:
tick = time.time()
ingredient = 'water'
for product in routine._routine:
    cosdna_id = Ingredient(ingredient).link_sync().cosdna_id
    if cosdna_id in product._cosdna_ids:
        print(product.name)
        
tock = time.time()
print(tock-tick)

neutrogena rapid clear stubborn acne spot gel
roc retinol correxion® eye cream 0.5oz(15ml)
body merry retinol surge moisturizer
1.3733494281768799
